In [ ]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import time
import warnings
import os
from tiki.mini import TikiMini

warnings.filterwarnings('ignore', category=UserWarning)
os.environ['GST_DEBUG'] = '0'

# 로봇 초기화
tiki = TikiMini()
tiki.set_motor_mode(tiki.MOTOR_MODE_PWM)
print("로봇 초기화 완료")

# 주행 파라미터
base_speed = 25          # 기본 전진 속도 (0~127)
max_steering = 12        # 최대 조향값 (값을 줄이면 회전 강도 감소)
frame_center_x = 160     # 프레임 중앙 X (320x240 해상도 기준)

# PID 제어기
class PIDController:
    def __init__(self, kp=0.6, ki=0.0, kd=0.15):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.integral = 0
        self.prev_error = 0

    def compute(self, error):
        self.integral += error
        derivative = error - self.prev_error
        self.prev_error = error

        # 적분 항 과도 누적 방지
        self.integral = np.clip(self.integral, -200, 200)

        out = self.kp * error + self.ki * self.integral + self.kd * derivative
        return out

pid = PIDController()
print("PID 제어기 초기화 완료")

# 카메라 파이프라인 (320x240, 30fps)
pipeline = (
    "nvarguscamerasrc ! "
    "video/x-raw(memory:NVMM), width=320, height=240, format=NV12, framerate=30/1 ! "
    "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! "
    "video/x-raw, format=BGR ! appsink"
)

print("카메라 초기화 중...")
cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
if not cap.isOpened():
    raise RuntimeError("카메라를 열 수 없습니다. 다른 노트북이 카메라를 점유 중인지 확인하세요.")

video_widget = widgets.Image(format='jpeg', layout=widgets.Layout(width='320px', height='240px'))
display(video_widget)
print("카메라/비디오 위젯 초기화 완료")


def frame_to_bytes(frame):
    _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 40])
    return buf.tobytes()


In [ ]:
def sliding_window_center(binary_mask, num_windows=8, margin=20, min_pixels=30):
    """슬라이딩 윈도우로 라인 중심 X를 계산 (없으면 None 반환)"""
    h, w = binary_mask.shape[:2]
    window_height = max(1, h // num_windows)

    ys, xs = np.where(binary_mask == 255)
    if len(xs) == 0:
        return None

    # 하단부에서 초기 중심 추정
    bottom_thresh = int(h * 0.75)
    bottom_inds = ys >= bottom_thresh
    if np.any(bottom_inds):
        current_x = int(np.mean(xs[bottom_inds]))
    else:
        current_x = int(np.mean(xs))

    centers = []

    for i in range(num_windows):
        win_y_low = h - (i + 1) * window_height
        win_y_high = h - i * window_height
        win_y_low = max(0, win_y_low)
        if win_y_high <= win_y_low:
            continue

        win_inds = (ys >= win_y_low) & (ys < win_y_high)
        win_xs = xs[win_inds]
        if len(win_xs) == 0:
            continue

        x_min = max(0, current_x - margin)
        x_max = min(w - 1, current_x + margin)
        lane_inds = (win_xs >= x_min) & (win_xs <= x_max)
        if np.sum(lane_inds) < min_pixels:
            continue

        current_x = int(np.mean(win_xs[lane_inds]))
        centers.append(current_x)

    if len(centers) == 0:
        return int(np.mean(xs))

    return int(np.mean(centers))


def detect_lane_center(frame_bgr,
                       yellow_lower=(20, 100, 100), yellow_upper=(30, 255, 255),
                       white_threshold=160):
    """ 
    양쪽 노란선을 경계로 삼아 가운데 흰색선을 찾고, 
    가로 흰줄(정지선/횡단선)은 직진용으로 판정하는 함수.

    Returns:
        center_x_frame: 라인 중심 X (프레임 좌표, 없으면 None)
        is_horizontal_bar: 가로 흰줄/넓은 흰 영역 여부 (True면 조향 없이 직진 권장)
    """
    h, w = frame_bgr.shape[:2]

    # 1. ROI: 하단 30~35%만 사용 (로봇과 가까운 부분만)
    roi_y1 = int(h * 0.65)
    roi_y2 = h
    roi_x1 = int(w * 0.10)
    roi_x2 = int(w * 0.90)
    roi = frame_bgr[roi_y1:roi_y2, roi_x1:roi_x2]
    roi_h, roi_w = roi.shape[:2]

    # 2. 노란선 마스크 (양쪽 차로 경계)
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    yellow_mask = cv2.inRange(hsv,
                              np.array(yellow_lower, dtype=np.uint8),
                              np.array(yellow_upper, dtype=np.uint8))

    roi_center_x = roi_w // 2
    left_x = None
    right_x = None

    if np.sum(yellow_mask == 255) > 100:
        left_half = yellow_mask[:, :roi_center_x]
        right_half = yellow_mask[:, roi_center_x:]

        ly, lx = np.where(left_half == 255)
        if len(lx) > 30:
            left_x = int(np.mean(lx))

        ry, rx = np.where(right_half == 255)
        if len(rx) > 30:
            right_x = int(np.mean(rx)) + roi_center_x

    # 3. 검색 영역: 노란선 사이 or 전체 ROI
    margin = 10
    search_x1 = 0
    search_x2 = roi_w

    if left_x is not None and right_x is not None and right_x - left_x > 2 * margin:
        search_x1 = left_x + margin
        search_x2 = right_x - margin
    elif left_x is not None:
        search_x1 = left_x + margin
    elif right_x is not None:
        search_x2 = right_x - margin

    search_x1 = max(0, search_x1)
    search_x2 = min(roi_w, search_x2)
    if search_x2 <= search_x1:
        search_x1, search_x2 = 0, roi_w

    # 4. 검색 영역 내 흰색선 마스크 생성
    search_roi = roi[:, search_x1:search_x2]
    gray = cv2.cvtColor(search_roi, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, white_mask = cv2.threshold(gray_blur, white_threshold, 255, cv2.THRESH_BINARY)

    ys, xs = np.where(white_mask == 255)
    search_w = search_x2 - search_x1
    search_h = roi_h

    if len(xs) == 0:
        return None, False

    # 5. 슬라이딩 윈도우로 중심 계산
    center_x_rel = sliding_window_center(white_mask)
    if center_x_rel is None:
        return None, False

    # ROI/프레임 좌표계로 변환
    center_x_roi = search_x1 + center_x_rel
    center_x_frame = roi_x1 + center_x_roi

    # 6. 가로 흰줄/넓은 흰 영역 판정
    x_min = int(np.min(xs))
    x_max = int(np.max(xs))
    x_range = x_max - x_min

    y_min = int(np.min(ys))
    y_max = int(np.max(ys))
    y_range = y_max - y_min

    white_count = len(xs)
    search_area = search_w * search_h

    is_wide = x_range > search_w * 0.6          # 가로로 넓게 퍼짐
    is_large = white_count > search_area * 0.15 # 면적이 충분히 큼
    is_thin_horizontal = y_range < search_h * 0.25  # 세로 두께가 얇음

    is_horizontal_bar = (is_wide and is_large) or (is_wide and is_thin_horizontal)

    return center_x_frame, is_horizontal_bar


In [ ]:
print("슬라이딩 윈도우 + 노란선 경계 기반 흰색선 추종 메인 루프 준비 완료")

print("기본 속도:", base_speed)
print("최대 조향값:", max_steering)

try:
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("[경고] 프레임 읽기 실패")
            time.sleep(0.1)
            continue

        frame = cv2.flip(frame, -1)  # 카메라 장착 방향에 따라 필요시 조정

        center_x, is_horizontal_bar = detect_lane_center(frame)

        if center_x is not None:
            error = center_x - frame_center_x

            # 에러 부호로 주행 방향 판단 (라인이 어느 쪽에 있는지)
            # error > 0  → 라인이 화면 오른쪽에 있음 → 로봇은 오른쪽으로 더 이동해야 함
            # error < 0  → 라인이 화면 왼쪽에 있음 → 로봇은 왼쪽으로 더 이동해야 함
            if error > 3:
                dir_text = "RIGHT"   # 오른쪽으로 더 이동
            elif error < -3:
                dir_text = "LEFT"    # 왼쪽으로 더 이동
            else:
                dir_text = "CENTER"  # 거의 중앙

            # PID 기반 조향값 계산
            steering = pid.compute(error)
            # 회전 강도 조절 (0~1 사이로 줄이면 조향이 부드러워짐)
            steering *= 0.5
            steering = float(np.clip(steering, -max_steering, max_steering))

            # 가로 흰줄/넓은 흰 영역이면 조향 없이 직진
            if is_horizontal_bar:
                steering = 0.0

            left_speed = base_speed + steering
            right_speed = base_speed - steering

            left_speed = int(np.clip(left_speed, 0, 127))
            right_speed = int(np.clip(right_speed, 0, 127))

            tiki.set_motor_power(tiki.MOTOR_LEFT, left_speed)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, right_speed)

            # 콘솔 로그 출력 (실시간 방향 확인용)
            print(f"[주행] err={error:.1f}, steer={steering:.1f}, dir={dir_text}, H_BAR={is_horizontal_bar}")

            # 디버그 표시용 오버레이
            h, w = frame.shape[:2]
            cv2.circle(frame, (int(center_x), int(h * 0.8)), 5, (0, 255, 0), -1)
            cv2.circle(frame, (frame_center_x, int(h * 0.8)), 5, (0, 0, 255), -1)
            cv2.line(frame,
                     (int(center_x), int(h * 0.8)),
                     (frame_center_x, int(h * 0.8)),
                     (255, 0, 0), 2)

            status_text = "H-BAR STRAIGHT" if is_horizontal_bar else "FOLLOW"
            cv2.putText(frame,
                        f"Dir:{dir_text} Err:{error:.1f} Steer:{steering:.1f} L:{left_speed} R:{right_speed}",
                        (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(frame, status_text, (5, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        (0, 255, 255) if is_horizontal_bar else (0, 255, 0), 2)
        else:
            # 라인을 못 찾으면 정지
            tiki.stop()
            cv2.putText(frame, "NO LINE - STOP", (5, 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # 비디오 위젯 업데이트
        video_widget.value = frame_to_bytes(frame)

        frame_count += 1
        if frame_count % 60 == 0:
            print(f"[프레임 {frame_count}] center_x={center_x}, horizontal_bar={is_horizontal_bar}")

        time.sleep(0.02)  # 약 50Hz 주기

except KeyboardInterrupt:
    print("\n사용자 중단: 주행 종료")
finally:
    print("\n정지 및 리소스 해제 중...")
    tiki.stop()
    if cap is not None:
        cap.release()
    print("완료")
